In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from __future__ import annotations

# Load the important classes
import random as rnd

from env.environment import OnlineFlexibleResourceAllocationEnv
from agents.resource_weighting_agent import ResourceWeightingAgent
from agents.task_pricing_agent import TaskPricingAgent

In [3]:
# Setup environment
test_env = OnlineFlexibleResourceAllocationEnv.make('../settings/basic_env_setting.json')

In [4]:
# Setup the agents
task_pricing_agents = [TaskPricingAgent('TPA {}'.format(agent_num)) for agent_num in range(10)]
resource_weighting_agents = [ResourceWeightingAgent('RWA {}'.format(agent_num)) for agent_num in range(10)]

In [5]:
# Reset the environment
state = test_env.reset()
test_env

Env State (0x2036ae2cfc0)
Auction Task -> Basic 2 Task (0x2035d250dc8) - Unassigned, Storage: 103, Comp: 33, Results data: 21, Auction time: 0, Deadline: 9
Servers
	Basic 0 Server - Storage cap: 377, Comp cap: 29, Bandwidth cap: 22, Tasks: []
	Basic 1 Server - Storage cap: 438, Comp cap: 17, Bandwidth cap: 22, Tasks: []
	Basic 2 Server - Storage cap: 321, Comp cap: 24, Bandwidth cap: 15, Tasks: []
	Basic 3 Server - Storage cap: 440, Comp cap: 30, Bandwidth cap: 23, Tasks: []

Unallocated tasks
	Basic 25 Task (0x2036ae33ca8) - Unassigned, Storage: 113, Comp: 36, Results data: 16, Auction time: 0, Deadline: 6
	Basic 30 Task (0x2036ae33f78) - Unassigned, Storage: 93, Comp: 56, Results data: 12, Auction time: 5, Deadline: 11
	Basic 8 Task (0x2036ae33318) - Unassigned, Storage: 99, Comp: 30, Results data: 23, Auction time: 7, Deadline: 14
	Basic 11 Task (0x2036ae334c8) - Unassigned, Storage: 119, Comp: 39, Results data: 29, Auction time: 8, Deadline: 20
	Basic 19 Task (0x2036ae33948) - Unas

In [6]:
state

Env State (0x2036ae2cfc0)
Auction Task -> Basic 2 Task (0x2035d250dc8) - Unassigned, Storage: 103, Comp: 33, Results data: 21, Auction time: 0, Deadline: 9
Servers
	Basic 0 Server - Storage cap: 377, Comp cap: 29, Bandwidth cap: 22, Tasks: []
	Basic 1 Server - Storage cap: 438, Comp cap: 17, Bandwidth cap: 22, Tasks: []
	Basic 2 Server - Storage cap: 321, Comp cap: 24, Bandwidth cap: 15, Tasks: []
	Basic 3 Server - Storage cap: 440, Comp cap: 30, Bandwidth cap: 23, Tasks: []


In [7]:
server_task_pricing_agents = {server: rnd.choice(task_pricing_agents) 
                              for server in state.server_tasks.keys()}
server_resource_allocation_agents = {server: rnd.choice(resource_weighting_agents) 
                                     for server in state.server_tasks.keys()}

In [8]:
auction_trajectory = []

def auction():
    actions = {
        server: server_task_pricing_agents[server].price(server, state.auction_task, 
                                                         state.server_tasks[server], env.time_step)
    }
    
    print('\n'.join([f'{str(server)} -> {price}' for server, price in actions.items()]))
    
    next_state, rewards, done, info = env.step(actions)
    
    auction_trajectory.append((state, rewards))
    
    print(f'Next State: {str(next_state)}')
    print(f'Rewards: {rewards}')
    print(f'Done: {done}')
    
    state = next_State

In [9]:
resource_allocation_trajectory = []

def resource_allocation():
    actions = {
        server: {
            task: server_resource_allocation_agents[server].weight(server, task, state.server_tasks[server], env.time_step)
            for task in tasks
        }
        for server, tasks in state.server_tasks.items()
    }
    
    print('\n'.join([f"{server.name} - [{', '.join([f'{task.name}: {actions[server][task]}'])}]"]))
    
    next_state, rewards, done, info = env.step(actions)
    
    resource_allocation_trajectory.append((state, next_state, rewards))
    
    print(f'Next State: {str(next_state)}')
    print(f'Rewards: {rewards}')
    print(f'Done: {done}')